In [2]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn  as sns
from scipy import stats
import math
import statistics
#讓圖可以在 jupyter notebook顯示
%matplotlib inline

df_train = pd.read_csv('HWData/Titanic_train.csv')

#觀察Age是否有異常值方法1 -- 先透過 計算統計值， 分別呈現數量，年齡平均，標準差，最大值和最小值。
print(df_train['Age'].describe()) #age平均

#年齡最小 0.4 歲和 80 歲都在年紀的合理範圍內

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64


In [8]:
#觀察Age是否有異常值方法2 --
# 創建一個函數，計算在這個資料中， ys:資料，times : 幾倍標準差，找出在這樣條件下的異常值。
def outliers_z_score(ys,times):
    mean_y = np.mean(ys)
    stdev_y = np.std(ys)
    z_scores = [(y - mean_y) / stdev_y for y in ys]
    return np.where(np.abs(z_scores) > times)

out_index=outliers_z_score(df_train['Age'],3)

print('z-score計算出3倍標準差以外的值有: ' )
print(df_train.loc[out_index[0],'Age'])


z-score計算出3倍標準差以外的值有: 
630    80.0
851    74.0
Name: Age, dtype: float64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys


In [9]:
#觀察Age是否有異常值方法3 --
#透過一個IQR函數來判斷是否為異常值。
def outliers_iqr(ys,times):
    #注意 使用 np.percentile 要去除 nan 要不然計算出來會錯誤，所以我們採用下者的程式 np.nanpercentile
    quartile_1, quartile_3 = np.nanpercentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * times)
    upper_bound = quartile_3 + (iqr * times)
    return np.where((ys > upper_bound) | (ys < lower_bound))

out_index2=outliers_iqr(df_train['Age'],1.5)
print(out_index2)
print('計算出3倍標準差以外的值有: (1.5 倍IQR)')
print(df_train.loc[out_index2[0],'Age'])

(array([ 33,  54,  96, 116, 280, 456, 493, 630, 672, 745, 851], dtype=int64),)
z-score計算出3倍標準差以外的值有: (1.5 倍IQR)
33     66.0
54     65.0
96     71.0
116    70.5
280    65.0
456    65.0
493    71.0
630    80.0
672    70.0
745    70.0
851    74.0
Name: Age, dtype: float64


In [10]:
out_index3=outliers_iqr(df_train['Age'],2)
print(out_index3)
print('計算出3倍標準差以外的值有: (2 倍IQR)')
print(df_train.loc[out_index3[0],'Age'])

(array([630, 851], dtype=int64),)
計算出3倍標準差以外的值有: (2 倍IQR)
630    80.0
851    74.0
Name: Age, dtype: float64


In [ ]:
#登船者的年齡不應設outlier,此年齡區間應都有可能；除非有數據輸入錯誤、故意離群(謊報年紀)的實據。